In [1]:
import numpy as np
import matplotlib.pyplot as plt
import Data
import pandas as pd
from os import sep as sep
import os
import matplotlib.cm as cm

In [ ]:
pd.__version__


In [2]:
root_path = 'Y:\Lior&Einav\Experiments'
exp_details = pd.read_excel(root_path + sep + 'Experiments_details.xlsx', engine='openpyxl')
exp_details = exp_details.drop(index=0)  # remove pilot experiment
exp_details

,Experiment,Larvae_present,Colony,P_yellow,C_yellow,P_red,C_red
1,11,yes,A,2,1,1,3
2,12,no,A,2,1,1,3
3,13,no,A,2,1,1,3
4,16,no,B,2,1,1,3
5,17,no,B,2,1,1,3
6,18,yes,A,2,1,1,3
7,19,yes,A,2,1,1,3
8,20,yes,A,2,1,1,3
9,21,no,B,2,1,1,3
10,22,no,B,2,1,1,6


In [3]:
exp_list = exp_details.Experiment.to_list()

In [4]:
def get_exp_path(exp_num, root_path):
    folderlist = os.listdir(root_path)
    exp_folder = [x for x in folderlist if x.startswith('experiment'+str(exp_num))]
    return root_path + sep + exp_folder[0]

In [5]:
def get_PC_ratios_dict(exp_details,row_idx):
    PC_ratios_dict = {}
    for color in ['yellow', 'red']:
        PC_ratios_dict[color] = [exp_details.loc[row_idx,'P_'+color], exp_details.loc[row_idx,'C_'+color]]
    return PC_ratios_dict

In [6]:
def plot_intake_trajectory(fdata, PC_ratios_dict, title=None):
    
    fdata[['P_mg','C_mg']] = fdata.apply(lambda x: pd.Series(Data.food_volume_to_PC_amounts(x['feeding_size_ul'],
                                                                                            PC_ratios_dict[x['food_source']])),
                                         axis=1)
    
    fdata = fdata.sort_values(by='feeding_start')
    fdata['cumP_mg'] = np.cumsum(fdata['P_mg'])
    fdata['cumC_mg'] = np.cumsum(fdata['C_mg'])

    #plt.figure()
    plt.rcParams.update({'font.size': 13})
    plt.figure(figsize=[4,4])
    plt.set_cmap(cm.Set1)
    m = 29
    n = 22
    x_p = PC_ratios_dict['yellow'][0]
    y_p = PC_ratios_dict['yellow'][1]
    x_c = PC_ratios_dict['red'][0]
    y_c = PC_ratios_dict['red'][1]
    plt.plot([0, m*x_p],[0, m*y_p],'k',linewidth=0.8)
    plt.plot([0,n*x_c],[0, n*y_c],'k',linewidth=0.8)
    plt.plot([0,fdata['P_mg'].sum()],[0,fdata['C_mg'].sum()],'--k',linewidth=1)
    X = np.insert(fdata['cumP_mg'].to_numpy(),0,0)[0:-1]
    Y = np.insert(fdata['cumC_mg'].to_numpy(),0,0)[0:-1]
    plt.quiver(X,Y,fdata['P_mg'],fdata['C_mg'],fdata.ant_id.astype("category").cat.codes,scale=1,scale_units='xy',
               angles='xy',width=0.015,alpha=1)
    plt.xlim([0,43])
    plt.ylim([0,43])
    plt.xlabel('Protein [mg]')
    plt.ylabel('Carbohydrates [mg]')
    plt.tight_layout()
    
    if title:
        plt.title(title)
        
    return fdata

In [34]:
def plot_convergence(fdata, ax, x_axis='num_feedings', title=None, same_fig=False):
    fdata = fdata.reset_index().sort_values(by='feeding_start')
    fdata['colony_PC_ratio'] = fdata['cumP_mg']/fdata['cumC_mg']
    final_ratio = fdata['colony_PC_ratio'].iloc[-1]
    
    if exp_details.loc[exp_idx,'Colony'] == 'A':
        c = 'b'
    elif exp_details.loc[exp_idx,'C_red'] == 6:
        c = 'g'
    else:
        c = 'm'
    
    if same_fig:
        plt.figure(999)
    else:
        plt.figure()
            
    if x_axis == 'num_feedings':
        h, = ax.plot(fdata['colony_PC_ratio']-final_ratio, c)
        ax.axhline(color ="k", linestyle ="--")
        ax.set_xlabel('number of feedings')
    elif x_axis == 'frame':
        h, = ax.plot(fdata['feeding_start'],fdata['colony_PC_ratio']-final_ratio, c)
        ax.axhline(color ="k", linestyle ="--")
        ax.set+xlabel('frame')
        
    ax.ylabel('difference from final intake ratio')
    
    if title:
        plt.title(title)
    
    return h, c

In [50]:
def plot_foragers_trajectories(fdata, PC_ratios_dict, ax, title=None):
    fdata_byforager=fdata.groupby(fdata['ant_id'])
    fdata['ant_cumC_mg'] = fdata_byforager['C_mg'].apply(np.cumsum)
    fdata['ant_cumP_mg'] = fdata_byforager['P_mg'].apply(np.cumsum)
    fdata = fdata.reset_index().sort_values(by='feeding_start')
    fdata.reset_index()
    
    
    X = fdata_byforager['ant_cumP_mg'].apply(lambda x: np.insert(x.to_numpy(),0,0)[0:-1]).to_list()
    X = np.concatenate(X).ravel()
    Y = fdata_byforager['ant_cumC_mg'].apply(lambda x: np.insert(x.to_numpy(),0,0)[0:-1]).to_list()
    Y = np.concatenate(Y).ravel()
    
    fdata=fdata.sort_values(by='ant_id')
    #plt.rcParams.update({'font.size': 13})
    #plt.figure(figsize=[4,4])
    plt.set_cmap(cm.Set1)
    m = 18
    n = 12
    x_p = PC_ratios_dict['yellow'][0]
    y_p = PC_ratios_dict['yellow'][1]
    x_c = PC_ratios_dict['red'][0]
    y_c = PC_ratios_dict['red'][1]
    ax.plot([0, m*x_p],[0, m*y_p],'k',linewidth=0.8)
    ax.plot([0,n*x_c],[0, n*y_c],'k',linewidth=0.8)
    ax.plot([0,fdata['P_mg'].sum()],[0,fdata['C_mg'].sum()],'--k',linewidth=1)
    #plt.plot(X,Y)
    ax.quiver(X,Y,fdata.sort_values(by=['ant_id','feeding_start'])['P_mg'],
               fdata.sort_values(by=['ant_id','feeding_start'])['C_mg'],
               fdata.ant_id.astype("category").cat.codes,scale=1,scale_units='xy',angles='xy',width=0.008,alpha=1)
    
    maxlim = max(fdata['ant_cumC_mg'].max(),fdata['ant_cumP_mg'].max())*1.1
    ax.set_xlim([0,maxlim])
    ax.set_ylim([0,maxlim])
    #plt.xlabel('Protein [mg]')
    #plt.ylabel('Carbohydrates [mg]')
    ax.set_aspect('equal', 'box')
    #plt.tight_layout()
    if title:
        plt.title(title)

In [27]:
def plot_accumulation_dynamics(fdata,title=None, fig=None, ax=None):
    if not fig:
        fig,ax = plt.subplots()
    fdata.reset_index()
    fdata['cum_vol'] = fdata.groupby(fdata['food_source'])['feeding_size_ul'].apply(np.cumsum)
    fdata['start_time'] = fdata['feeding_start']*1.4/60
    fdata.groupby(fdata['food_source']).plot('start_time','cum_vol',ax=ax)
    plt.xlabel('time [min.]')
    plt.ylabel('Volume consumed [ul]')
    plt.legend(['sugar-rich','protein-rich'])
    if title:
        plt.title(title)

In [ ]:
%matplotlib qt
legend_dict = {'b':'with larvae 3:1 vs. 1:2', 'm':'no larvae 3:1 vs. 1:2','g':'no larvae 6:1 vs. 1:2'}
existing_colors=[]
for exp_num in exp_list:
    exp_path = get_exp_path(exp_num, root_path)
    
    exp_idx = exp_details.index[exp_details.Experiment==exp_num][0]
    PC_ratios_dict = get_PC_ratios_dict(exp_details,exp_idx)
    
    fdata = pd.read_csv(exp_path + sep + 'forager_table_with_feeding_sizes_ul_transparency_corrected.csv')
    
    fdata = plot_intake_trajectory(fdata, PC_ratios_dict, 'exp '+str(exp_num))
    plt.close()
    
    plot_foragers_trajectories(fdata, PC_ratios_dict, 'exp '+str(exp_num))
    #plt.close()
    
#     h, c = plot_convergence(fdata, title='exp '+str(exp_num), same_fig=False)
#     if c not in existing_colors:
#         h.set_label(legend_dict[c])
#         existing_colors.append(c)
#     #plt.close()
    
    #fig, ax = plt.subplots()
    #plot_accumulation_dynamics(fdata, title='exp '+str(exp_num))

In [ ]:
plt.close('all')

In [ ]:
42/3


In [10]:
def get_exp_data(exp_num, exp_details, root_path):
    exp_path = get_exp_path(exp_num, root_path)
    exp_idx = exp_details.index[exp_details.Experiment==exp_num][0]
    PC_ratios_dict = get_PC_ratios_dict(exp_details,exp_idx)
    fdata = pd.read_csv(exp_path + sep + 'forager_table_with_feeding_sizes_ul_transparency_corrected.csv')
    
    return exp_idx, PC_ratios_dict, fdata, exp_path

In [11]:
larvae_present = [11, 18, 19]
no_larvae = [16, 17, 21]

In [32]:
%matplotlib qt
fig, ax = plt.subplots(2,3, sharex=True, sharey=True)
counter = 0
for exp_num in larvae_present:
    exp_idx, PC_ratios_dict, fdata, exp_path = get_exp_data(exp_num, exp_details, root_path)
    plot_accumulation_dynamics(fdata,title=None, fig=fig, ax=ax[0,counter])
    ax[0,counter].get_legend().remove()
    ax[0,counter].set_xlabel('')
    ax[0,counter].set_ylabel('')
    counter += 1
    
counter = 0
for exp_num in no_larvae:
    exp_idx, PC_ratios_dict, fdata, exp_path = get_exp_data(exp_num, exp_details, root_path)
    plot_accumulation_dynamics(fdata,title=None, fig=fig, ax=ax[1,counter])
    ax[1,counter].get_legend().remove()
    ax[1,counter].set_xlabel('')
    ax[1,counter].set_ylabel('')
    counter += 1

C:\Users\liorba\Anaconda3\lib\site-packages\pandas\plotting\_matplotlib\tools.py:331: MatplotlibDeprecationWarning: 
The is_first_col function was deprecated in Matplotlib 3.4 and will be removed two minor releases later. Use ax.get_subplotspec().is_first_col() instead.
  if ax.is_first_col():
C:\Users\liorba\Anaconda3\lib\site-packages\pandas\plotting\_matplotlib\tools.py:331: MatplotlibDeprecationWarning: 
The is_first_col function was deprecated in Matplotlib 3.4 and will be removed two minor releases later. Use ax.get_subplotspec().is_first_col() instead.
  if ax.is_first_col():
C:\Users\liorba\Anaconda3\lib\site-packages\pandas\plotting\_matplotlib\tools.py:331: MatplotlibDeprecationWarning: 
The is_first_col function was deprecated in Matplotlib 3.4 and will be removed two minor releases later. Use ax.get_subplotspec().is_first_col() instead.
  if ax.is_first_col():
C:\Users\liorba\Anaconda3\lib\site-packages\pandas\plotting\_matplotlib\tools.py:331: MatplotlibDeprecationWarning:

In [51]:
fig, ax = plt.subplots(2,3)
counter=0
for exp_num in larvae_present:
    exp_idx, PC_ratios_dict, fdata, exp_path = get_exp_data(exp_num, exp_details, root_path)
    fdata[['P_mg','C_mg']] = fdata.apply(lambda x: pd.Series(Data.food_volume_to_PC_amounts(x['feeding_size_ul'],
                                                                                            PC_ratios_dict[x['food_source']])),
                                         axis=1)
    
    fdata = fdata.sort_values(by='feeding_start')
    fdata['cumP_mg'] = np.cumsum(fdata['P_mg'])
    fdata['cumC_mg'] = np.cumsum(fdata['C_mg'])
    plot_foragers_trajectories(fdata, PC_ratios_dict, ax[0,counter])
    counter += 1
    
counter=0
for exp_num in no_larvae:
    exp_idx, PC_ratios_dict, fdata, exp_path = get_exp_data(exp_num, exp_details, root_path)
    fdata[['P_mg','C_mg']] = fdata.apply(lambda x: pd.Series(Data.food_volume_to_PC_amounts(x['feeding_size_ul'],
                                                                                            PC_ratios_dict[x['food_source']])),
                                         axis=1)
    
    fdata = fdata.sort_values(by='feeding_start')
    fdata['cumP_mg'] = np.cumsum(fdata['P_mg'])
    fdata['cumC_mg'] = np.cumsum(fdata['C_mg'])
    plot_foragers_trajectories(fdata, PC_ratios_dict, ax[1,counter])
    counter += 1

In [24]:
fdata['start_time'] = fdata['feeding_start']*1.4/60
fdata.head()

,Unnamed: 0,ant_id,crop_after,crop_before,feeding_end,feeding_size_intensity,feeding_start,first_interaction_after_partner,first_interaction_after_start,food_source,last_interaction_before_end,last_interaction_before_partner,feeding_size_ul,cum_vol,start_time
0,0,23,442727.773438,0.0000,88,442727.773438,8,NaN,NaN,yellow,NaN,NaN,5.270962,5.270962,0.186667
1,1,208,115213.312500,0.0000,46,115213.312500,21,NaN,NaN,yellow,NaN,NaN,5.228153,10.499115,0.490000
2,2,203,217490.921875,0.0000,55,217490.921875,27,NaN,NaN,yellow,NaN,NaN,2.417470,12.916585,0.630000
3,3,169,223934.937500,0.0000,107,223934.937500,42,NaN,NaN,yellow,NaN,NaN,3.669312,16.585896,0.980000
4,4,208,319361.618909,115213.3125,96,204148.306409,67,NaN,NaN,yellow,NaN,NaN,9.263847,25.849743,1.563333
